# Import data, EDA

### Import and filtering

In [48]:
import numpy as np
from tabulate import tabulate
import pandas as pd


# Importing data from csv file provided to df
file_path = 'ironkaggle_real_regressions.csv'
ironkaggle_df = pd.read_csv(file_path)

# Display the first few rows of the DataFrame
# print(ironkaggle_df.head())
print(ironkaggle_df.dtypes)
# print(ironkaggle_df.describe())
print(ironkaggle_df.columns)


date                       object
index                       int64
store_ID                    int64
day_of_week                 int64
nb_customers_on_day         int64
open                        int64
promotion                   int64
state_holiday              object
school_holiday              int64
predicted_sales_xgboost    object
dtype: object
Index(['date', 'index', 'store_ID', 'day_of_week', 'nb_customers_on_day',
       'open', 'promotion', 'state_holiday', 'school_holiday',
       'predicted_sales_xgboost'],
      dtype='object')


In [49]:
# <!-- | **Variable**           | **Type**         | **Details**                              |
# |-------------------------|------------------|------------------------------------------|
# | `store_ID`             | Categorical     | Nominal, unique store identifiers        |
# | `day_of_week`          | Categorical     | Represents days of the week   |
# | `date`                 | Ordinal         | Nominal (date) |
# | `nb_customers_on_day`  | Numerical       | number of customers          |
# | `open`                 | Categorical     | Binary, open or closed         |
# | `promotion`            | Categorical     | Binary, promotion status                 |
# | `state_holiday`        | Categorical     | Binary, holiday status                  |
# | `school_holiday`       | Categorical     | Binary, holiday status                   |
# | `predicted_sales_xgboost`                | Numerical       | Continuous, sales revenue                | 

categorical_var = ['store_ID', 'day_of_week', 'open', 'promotion', 'state_holiday', 'school_holiday']
date_var = ['date']
numerical_var = ['nb_customers_on_day', 'predicted_sales_xgboost']

# I will drop the 'index' and  
ironkaggle_df = ironkaggle_df.drop(columns=['index'])

# Converting the 'date' column to datetime
ironkaggle_df['date'] = pd.to_datetime(ironkaggle_df['date'])

# Preprocess the data
ironkaggle_df["predicted_sales_xgboost"] = (
    ironkaggle_df["predicted_sales_xgboost"]
    .replace(",", "", regex=True)  # Remove commas
    .replace(r"[^\d.]", np.nan, regex=True)  # Replace non-numeric characters with NaN
    .astype(float)  # Convert to float
)

# Count the number of unique values in a column
unique_values = ironkaggle_df['state_holiday'].unique()
print(f"Unique values in the column: {unique_values}")

# Remapping 'state_holiday'
# Unique values in the column: ['0' 'a' 'c' 'b'] converting to ['0' '1' '2 '3']. 
# Define the mapping
mapping = {'0': '0', 'a': '1', 'c': '2', 'b': '3'}

# Apply the mapping to the column
ironkaggle_df['state_holiday'] = ironkaggle_df['state_holiday'].map(mapping)
# converting to int
ironkaggle_df['state_holiday'] = ironkaggle_df['state_holiday'].astype('int64')

# Categorical var
for col in categorical_var:
    unique_count = ironkaggle_df[col].nunique()
    print(f"The column '{col}' has {unique_count} unique values.")

Unique values in the column: ['0' 'a' 'b' 'c']
The column 'store_ID' has 1115 unique values.
The column 'day_of_week' has 7 unique values.
The column 'open' has 2 unique values.
The column 'promotion' has 2 unique values.
The column 'state_holiday' has 4 unique values.
The column 'school_holiday' has 2 unique values.


In [50]:
# Filter rows where 'open' is 0
closed_sales = ironkaggle_df[ironkaggle_df['open'] == 0]['predicted_sales_xgboost']

# Summary statistics for 'sales' when 'open' is 0
closed_sales_summary = closed_sales.describe()
# print(closed_sales_summary)

# There are no sales when stores are closed. These rows do not provide insights for sales analysis
# I will remove them from the dataset. 

# Remove rows where 'open' is 0 and 'sales' is also 0
ironkaggle_df_filtered = ironkaggle_df[~((ironkaggle_df['open'] == 0) & (ironkaggle_df['predicted_sales_xgboost'] == 0))]
# Drop the entire 'open' feature
ironkaggle_df_filtered = ironkaggle_df_filtered.drop(columns=['open'])
categorical_var.remove('open')



## EDA

In [51]:
# For the numerical var, what are their descriptive statistics?
# For the Categorical var, what are their counts? 


def calculate_descriptive_stats(df, column):
    """
    Calculates descriptive statistics for a given column of a DataFrame.

    Parameters:
        df (pd.DataFrame): The DataFrame containing the data.
        column (str): The column name for which to calculate statistics.

    Returns:
        pd.DataFrame: A DataFrame containing the calculated statistics.
    """
    # Calculate statistics
    stats = {
        "Statistic": [
            "Mean",
            "Median",
            "Mode",
            "Range",
            "Variance",
            "Standard Deviation",
            "Interquartile Range (IQR)",
            "Skewness",
            "Kurtosis",
            "Minimum",
            "Maximum",
            "Sum",
            "Count",
            "25th Percentile",
            "75th Percentile",
        ],
        "Value": [
            df[column].mean(),
            df[column].median(),
            df[column].mode()[0] if not df[column].mode().empty else np.nan,
            df[column].max() - df[column].min(),
            df[column].var(),
            df[column].std(),
            df[column].quantile(0.75) - df[column].quantile(0.25),
            df[column].skew(),
            df[column].kurt(),
            df[column].min(),
            df[column].max(),
            df[column].sum(),
            df[column].count(),
            df[column].quantile(0.25),
            df[column].quantile(0.75),
        ],
    }

    # Create a DataFrame
    stats_df = pd.DataFrame(stats)

    # Print neatly using tabulate
    # print(tabulate(stats_df, headers='keys', tablefmt='grid', showindex=False))

    return stats_df

def calculate_stats_for_multiple_columns(df, columns):
    """
    Calculates descriptive statistics for multiple columns and stores them in a dictionary.

    Parameters:
        df (pd.DataFrame): The DataFrame containing the data.
        columns (list): List of column names to calculate statistics for.

    Returns:
        dict: A dictionary where keys are column names and values are DataFrames of statistics.
    """
    results = {}
    for column in columns:
        # print(f"\nDescriptive Statistics for {column}:")
        stats_df = calculate_descriptive_stats(df, column)
        results[column] = stats_df
    return results

# Example Usage
# Assuming `df` is your DataFrame and `numerical_var` is a list of numerical columns
numerical_var = ['nb_customers_on_day', 'predicted_sales_xgboost']
results = calculate_stats_for_multiple_columns(ironkaggle_df_filtered, numerical_var)

# grouping by store
# Group 'sales' by 'store_ID' and compute aggregate statistics
grouped_sales = ironkaggle_df_filtered.groupby('store_ID')['predicted_sales_xgboost'].agg(['sum', 'mean', 'median', 'count', 'max', 'min']).reset_index()

# Display the grouped DataFrame
# print(grouped_sales)

In [52]:
import matplotlib.pyplot as plt

# Plot sum and means to see distributions
# Extracting data for plotting
store_count = len(grouped_sales["store_ID"])
sums = grouped_sales["sum"]
mean = grouped_sales["mean"]

# Convert sums to millions and means to thousands
sums_in_millions = [x / 1_000_000 for x in sums]
means_in_thousands = [x / 1_000 for x in mean]

# Create a single figure for all 4 plots
plt.figure(figsize=(14, 10))

# Histogram for sum of sales
plt.subplot(2, 2, 1)
n, bins, patches = plt.hist(sums_in_millions, bins=10, alpha=0.8, edgecolor='black')
for i in range(len(n)):
    plt.text((bins[i] + bins[i + 1]) / 2, n[i], str(int(n[i])), ha='center', va='bottom')
plt.title("Histogram of Sum of Predicted Sales (in Millions)")
plt.xlabel("Sum of Predicted Sales (in Millions)")
plt.ylabel("Store Count")
plt.grid(axis='y', linestyle='--', alpha=0.7)

# Histogram for mean sales
plt.subplot(2, 2, 2)
n, bins, patches = plt.hist(means_in_thousands, bins=10, alpha=0.8, edgecolor='black', color='orange')
for i in range(len(n)):
    plt.text((bins[i] + bins[i + 1]) / 2, n[i], str(int(n[i])), ha='center', va='bottom')
plt.title("Histogram of Mean Predicted Sales (in Thousands)")
plt.xlabel("Mean Predicted Sales (in Thousands)")
plt.ylabel("Store Count")
plt.grid(axis='y', linestyle='--', alpha=0.7)

# Box plot for sum of sales
plt.subplot(2, 2, 3)
plt.boxplot(sums, vert=False, patch_artist=True, boxprops=dict(facecolor='lightblue'))
plt.title("Box Plot of Sum of Predicted Sales")
plt.xlabel("Sum of Predicted Sales (in Dollars)")

# Box plot for mean of sales
plt.subplot(2, 2, 4)
plt.boxplot(mean, vert=False, patch_artist=True, boxprops=dict(facecolor='lightgreen'))
plt.title("Box Plot of Mean Predicted Sales")
plt.xlabel("Mean Predicted Sales (in Dollars)")

plt.tight_layout()
# plt.show()

# Save the plot to png
plt.savefig("figures/1_sales_distribution_summary_real_data.png", dpi=300, bbox_inches="tight")  
plt.close()  # Close the figure to prevent it from displaying in Jupyter

### Correlation Matrix

In [53]:
import seaborn as sns

correlation_matrix = ironkaggle_df_filtered.select_dtypes(include=['float64', 'int64']).corr()

# SVisualize the correlation matrix using a heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(
    correlation_matrix,
    annot=True,        # Display correlation values on the heatmap
    cmap="coolwarm",   # Color map for visualization
    fmt=".2f",         # Format to 2 decimal places
    linewidths=0.5     # Add space between cells
)
plt.title("Correlation Matrix Heatmap for predicted numerical var")

# Save plot to png
plt.savefig("figures/2_Correlation_Matrix_Heatmap_for_numerical_var_real_data.png.png", dpi=300, bbox_inches="tight")
plt.close() # Close the figure to prevent it from displaying in Jupyter
#plt.show()


